In [1]:
import pandas as pd
import numpy as np
from typing import List
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv('../data/interim/user-lisetning-records-2008-march.csv', index_col=[0])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482386 entries, 10470 to 19097502
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   userid                 482386 non-null  object
 1   timestamp              482386 non-null  object
 2   musicbrainz_artist_id  467352 non-null  object
 3   artist_name            482386 non-null  object
 4   musicbrainz_track_id   426429 non-null  object
 5   track_name             482386 non-null  object
 6   datetime               482386 non-null  object
 7   year                   482386 non-null  int64 
 8   month                  482386 non-null  int64 
 9   day                    482386 non-null  int64 
 10  hour                   482386 non-null  int64 
 11  artist_track           482386 non-null  object
dtypes: int64(4), object(8)
memory usage: 47.8+ MB


In [6]:
df.head()

,userid,timestamp,musicbrainz_artist_id,artist_name,musicbrainz_track_id,track_name,datetime,year,month,day,hour,artist_track
10470,user_000001,2008-03-31T14:21:08Z,NaN,Woodblue,NaN,Green Flash,2008-03-31 14:21:08+00:00,2008,3,31,14,Woodblue_Green Flash
10471,user_000001,2008-03-31T14:16:53Z,NaN,Woodblue,NaN,Iioa,2008-03-31 14:16:53+00:00,2008,3,31,14,Woodblue_Iioa
10472,user_000001,2008-03-31T14:02:39Z,NaN,Lushlife & The Age Of Imagination Quartet,NaN,No Foundation,2008-03-31 14:02:39+00:00,2008,3,31,14,Lushlife & The Age Of Imagination Quartet_No F...
10473,user_000001,2008-03-31T13:18:56Z,630662ea-1c7d-4208-99fd-ba3afec20f0c,Amon Tobin,3785cd62-4f04-49ee-a2ab-587956152891,Bloodstone,2008-03-31 13:18:56+00:00,2008,3,31,13,Amon Tobin_Bloodstone
10474,user_000001,2008-03-31T13:09:58Z,7c158ea8-c0aa-410e-bdc1-20bba9759577,The Cinematic Orchestra,919b8a05-e1eb-4402-a2a8-7cc40fcede7c,To Build A Home,2008-03-31 13:09:58+00:00,2008,3,31,13,The Cinematic Orchestra_To Build A Home


In [8]:
def show_stats(df:pd.DataFrame):
    """
    Show stats data
    @param df: dataframe
    """
    # how many unique users are there
    print("Unique users number: %d"%len(df.userid.unique()))
    # how many unique artists are there
    print("Unique artists number: %d"%len(df.artist_name.unique()))
    # how many unique songs are there
    print("Unique track number: %d"%len(df.track_name.unique()))
    # how many unique songs with artist are there
    if('artist_track' in df.columns):
        print("Unique artists_track number: %d"%len(df.artist_track.unique()))

In [9]:
show_stats(df)

Unique users number: 590
Unique artists number: 29197
Unique track number: 148659
Unique artists_track number: 178423


In [17]:
def construct_user_matrix(_df:pd.DataFrame) -> pd.DataFrame:
    """
    Construct a n*m dataframe where n means the number of users
    and m means the number of unique tracks.
    Each value means how many times the user listened to the certain track
    """

    df = _df.copy()

    # sort the orginal dataframe by userid
    df_sorted = df.sort_values('userid')
    df_sorted.reset_index(inplace=True)

    # one hot encodings for each track
    # set data type to int8 to reduce memory usage
    enc = OneHotEncoder(handle_unknown='ignore',dtype='int8')
    enc_df = enc.fit_transform(df_sorted[['artist_track']]).toarray()

    # get the list of occurrences of users
    user_list = list(df_sorted.userid.unique())
    user_occurrence_list = df_sorted.userid.value_counts()

    # user range dictionary with tuple of the start index and end index
    start = 0
    user_range_dic = {}
    for user in user_list:
        end = start+user_occurrence_list.loc[user]
        user_range_dic[user] = (start, end)
        start = end

    # initialize the array with all zeros
    # set data type to int instead of the default float to reduce memory usage
    user_track_arr = np.zeros((len(user_list), enc_df.shape[1]), dtype=int)

    # construct the final dataframe by summing up
    row = 0
    for user, (start, end) in tqdm(user_range_dic.items()):

        # sum the occurrences of each track given the range of rows for the user
        user_track_row = np.sum(enc_df[start:end], axis=0)

        # add it to numpy array
        user_track_arr[row,:] = user_track_row

        row += 1

    # column names excluding the 'artist_track_' prefix
    column_names = [i[13:] for i in enc.get_feature_names_out()]
    
    # convert the numpy array to dataframe
    user_track_df = pd.DataFrame(user_track_arr, index=user_list, columns=column_names)

    return user_track_df

In [18]:
user_track_df = construct_user_matrix(df)

100%|██████████| 590/590 [01:32<00:00,  6.37it/s]


In [19]:
user_track_df.head()

,"! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_05. Astrolog","! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_07. Syn Marnotrawny","! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_16. Pejzaz Zimowy","! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_17. Koniec Wojny 30-Letniej","! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_20. Rozmowa","! Www.Polskie-Mp3.Tk ! Jacek Kaczmarski, Gintrowski And Lapinski_21. Kantyczka Z Lotu Ptaka",!!!_A New Name,!!!_All My Heroes Are Weirdos,!!!_Bend Over Beethoven,!!!_Bend Over Beethoven (Original Nashville Jam),...,黒夢_Let'S Dance,黒夢_少年 -Screw Mix-,齋藤彩夏_Doki Doki Waku Waku,齋藤彩夏_ドキドキ☆ワクワク♪,서태지_I'M Gonna Forget You,소영_Turn Right At Midnight,유승준_Shut Up!!,이병우_Starving Brothers,이수영_Hatred,Ｄｊ Ｋａｗａｓａｋｉ_Ｌｉｋｅ Ｔｈｕｎｄｅｒ
user_000001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
user_000002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
user_000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
user_000004,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
user_000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## User track data frame persistence

In [20]:
user_track_df.to_csv('../data/interim/user_track_df.csv')

# Generate user listen count dataframe

In [20]:
count_df = df.groupby(['userid', 'artist_track']).count().reset_index()[['userid', 'artist_track', 'timestamp']]
count_df.rename(columns = {'timestamp':'listen_count'}, inplace = True)
count_df.to_csv('../data/interim/user-listen-count.csv')